In [41]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from affinity.load_data.scraper import scrape, scraper
import requests
import xmltodict
from sklearn.cluster import KMeans

## scrape & clean data

In [160]:
sitemap_urls = [
    "https://www.houseandgarden.co.uk/sitemap.xml?year=2024&month=6&week=1",
    "https://www.vanityfair.com/sitemap.xml?year=2024&month=6&week=1",
    "https://www.epicurious.com/sitemap.xml?year=2024&month=6&week=1",
    "https://www.epicurious.com/sitemap.xml?year=2024&month=5&week=3",
    "https://www.cntraveller.com/sitemap.xml?year=2024&month=5&week=5",
    "https://www.voguebusiness.com/sitemap.xml?year=2024&month=5&week=5",
    "https://www.voguebusiness.com/sitemap.xml?year=2024&month=5&week=3",
    "https://pitchfork.com/sitemap.xml?year=2024&month=5&week=4",
    "https://www.self.com/sitemap.xml?year=2024&month=6&week=1",
    "https://www.self.com/sitemap.xml?year=2024&month=4&week=3",
    "https://www.worldofinteriors.com/sitemap.xml?year=2024&month=2&week=3",
    "https://www.worldofinteriors.com/sitemap.xml?year=2024&month=5&week=4",
    "https://www.worldofinteriors.com/sitemap.xml?year=2023&month=6&week=2",
    "https://www.architecturaldigest.com/sitemap.xml?year=2024&month=6&week=1",
    "https://www.newyorker.com/sitemap.xml?year=2024&month=6&week=1",
    "https://www.tatler.com/sitemap.xml?year=2024&month=6&week=1"
]

In [161]:
# List of base sitemap URLs with placeholders for year, month, and week
base_sitemap_urls = [
    "https://www.houseandgarden.co.uk/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.vanityfair.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.epicurious.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.epicurious.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.cntraveller.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.voguebusiness.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.voguebusiness.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://pitchfork.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.self.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.self.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.worldofinteriors.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.worldofinteriors.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.worldofinteriors.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.architecturaldigest.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.newyorker.com/sitemap.xml?year={year}&month={month}&week={week}",
    "https://www.tatler.com/sitemap.xml?year={year}&month={month}&week={week}"
]

# Generate combinations of URLs by changing year, month, and week
sitemap_urls = []
for year in range(2020, 2025):
    for month in range(1, 13):
        for week in range(1, 5):
            for base_url in base_sitemap_urls:
                sitemap_urls.append(base_url.format(year=year, month=month, week=week))

# Print the generated sitemap URLs
for url in sitemap_urls:
    print(url)


https://www.houseandgarden.co.uk/sitemap.xml?year=2020&month=1&week=1
https://www.vanityfair.com/sitemap.xml?year=2020&month=1&week=1
https://www.epicurious.com/sitemap.xml?year=2020&month=1&week=1
https://www.epicurious.com/sitemap.xml?year=2020&month=1&week=1
https://www.cntraveller.com/sitemap.xml?year=2020&month=1&week=1
https://www.voguebusiness.com/sitemap.xml?year=2020&month=1&week=1
https://www.voguebusiness.com/sitemap.xml?year=2020&month=1&week=1
https://pitchfork.com/sitemap.xml?year=2020&month=1&week=1
https://www.self.com/sitemap.xml?year=2020&month=1&week=1
https://www.self.com/sitemap.xml?year=2020&month=1&week=1
https://www.worldofinteriors.com/sitemap.xml?year=2020&month=1&week=1
https://www.worldofinteriors.com/sitemap.xml?year=2020&month=1&week=1
https://www.worldofinteriors.com/sitemap.xml?year=2020&month=1&week=1
https://www.architecturaldigest.com/sitemap.xml?year=2020&month=1&week=1
https://www.newyorker.com/sitemap.xml?year=2020&month=1&week=1
https://www.tatler

In [162]:
len(sitemap_urls)

3840

In [163]:
def fetch_sitemap_urls(sitemap_url):
    try:
        response = requests.get(sitemap_url)
        if response.status_code == 200:

            dict_data = xmltodict.parse(response.content)

            urls = [entry['loc'] for entry in dict_data['urlset']['url']]

            return urls
    except:
        return []
    else:
        print(f"Failed to fetch {sitemap_url}: Status code {response.status_code}")
    return []

In [164]:
fetch_sitemap_urls(sitemap_urls[0])

['https://www.houseandgarden.co.uk/gallery/inside-the-sets-of-this-years-golden-globewinning-productions',
 'https://www.houseandgarden.co.uk/article/things-to-put-in-frames-isabella-cotier',
 'https://www.houseandgarden.co.uk/article/things-to-put-in-frames-rose-electra-harris',
 'https://www.houseandgarden.co.uk/gallery/february-2020-house-garden',
 'https://www.houseandgarden.co.uk/article/unfashionable-beauties-exploring-the-conifer',
 'https://www.houseandgarden.co.uk/gallery/bryan-osullivan-house',
 'https://www.houseandgarden.co.uk/recipe/endive-salad-with-charred-leeks-and-thyme-oil',
 'https://www.houseandgarden.co.uk/article/new-picture-of-the-royal-family',
 'https://www.houseandgarden.co.uk/gallery/ten-design-trends-that-defined-the-past-decade',
 'https://www.houseandgarden.co.uk/recipe/bridget-jones-pan-fried-salmon',
 'https://www.houseandgarden.co.uk/article/decimo-restaurant-review',
 'https://www.houseandgarden.co.uk/article/how-to-be-creative-with-space-in-the-kitche

In [165]:
list_urls = []
# Iterate over each sitemap URL and extract the URLs
for sitemap_url in sitemap_urls:
    list_urls = []
    urls = fetch_sitemap_urls(sitemap_url)
    list_urls.extend(urls)

In [ ]:
#save list of urls as csv
list_urls.to_csv("list_urls")
inventory_urls= pd.DataFrame()
inventory_urls["urls"]=list_urls
inventory_urls.to_csv("/Users/martafillolbruguera/code/affinity_at_scale/data/inventory_urls.csv",index=False)

In [ ]:
#import inventory_urls to scrape

scraped_inventory = pd.read_csv("/Users/martafillolbruguera/code/affinity_at_scale/data/inventory_urls.csv")

scraped_inventory["texts"] = scraped_inventory["urls"].apply(scraper)
scraped_inventory

In [ ]:
# data = pd.read_csv("/Users/martafillolbruguera/code/affinity_at_scale/data/urls.csv", sep=",")
# data = data.drop(columns=["Unnamed: 1","Unnamed: 2"])
# sample = data.sample(fraic=0.2)

# def scrape_with_exc(url):
#     url = "https://" + url
#     try:
#         text = scraper(url)
#     except:
#         text = ""
#     return text

# sample["Texts"] = sample["Unnamed: 3"].apply(scrape_with_exc)

/Users/martafillolbruguera/code/affinity_at_scale/affinity/load_data/scraper.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scraped_data = pd.Series(scraped_data)


In [ ]:
# sample.to_csv("20pct_scraped_data.csv")

In [ ]:
clean_texts = scraped_inventory.loc[scraped_inventory.texts != ""]

In [ ]:
#clean, tokenize, lemmatize

def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    cleaned = ' '.join(lemmatized) # Join back to a string
    return cleaned

# Apply to all texts
clean_texts['clean_text'] = clean_texts.texts.apply(clean)

clean_texts.head()

/var/folders/4p/pt0m2ypd27q_p574g81dytpw0000gn/T/ipykernel_33277/751833470.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_texts['clean_text'] = clean_texts.Texts.apply(clean)


,Site / App ID,Unnamed: 3,Texts,clean_text
50,football365.com,90min.com,Antonio Conte returns to management with job i...,antonio conte return management job serie psg ...
150,allgigs.co.uk,buzzfeed.com,Olivia Munn Spoke About Her Surprise First Pre...,olivia munn spoke surprise first pregnancy boy...
198,go.com,thecomet.net,Heartbroken dad hopes to help raise £ 100k in ...,heartbroken dad hope help raise two year old d...
714,NaN,wharfedaleobserver.co.uk,City centre style bar and restaurant set to op...,city centre style bar restaurant set open otle...
391,myrecipes.com,marieclaire.com,Layer Up Your Bangles This Summer Summer Outfi...,layer bangle summer summer outfit formula stay...


## LDA model

In [17]:
#LDA MODEL

vectorizer = CountVectorizer()
data_vectorized = vectorizer.fit_transform(clean_texts['clean_text'])
lda_model = LatentDirichletAllocation(n_components=30)
lda_vectors = lda_model.fit_transform(data_vectorized)

In [18]:
# ???

def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
print_topics(lda_model, vectorizer)

Topic 0:
[('london', 27.033333333333058), ('home', 22.715714136180758), ('sale', 22.030468333788296), ('stevenage', 16.033333333331825), ('new', 15.112015524493458), ('house', 14.933006935108434), ('channel', 12.033333333332937), ('partner', 12.03333333333293), ('market', 9.852206412974006), ('business', 9.361853047567507)]
Topic 1:
[('may', 22.725842381591136), ('best', 20.86643127254148), ('kid', 19.92678185688077), ('published', 18.999239808443804), ('last', 15.653465464671395), ('parent', 15.45223010227087), ('updated', 15.159021134116331), ('child', 13.552408735827676), ('recipe', 11.951821948258642), ('expert', 11.650003816786176)]
Topic 2:
[('salad', 134.0333333333328), ('chicken', 37.67988784715205), ('recipe', 37.03333333333211), ('wore', 23.033333333332635), ('potato', 21.85692210683672), ('cheese', 21.03333333333265), ('feta', 20.03333333333264), ('dressing', 19.03333333333264), ('make', 18.45308701300499), ('creamy', 17.03333333333182)]
Topic 3:
[('club', 11.033333333333301

In [ ]:
#apply the model on new data

new_data = ["Tiaras are the fashion accessory for high society brides,but how to wear them correctly? While elegant brides across the nation plan their upcoming nuptials, Tatler answers your questions on exactly how best to sport the regal accessory."]

new_data_vectorized = vectorizer.transform(new_data) #vectorize first
lda_vectors = lda_model.transform(new_data_vectorized) #transform using lda model fitted with our inventory_urls

print("topic 0 :", lda_vectors[0][0])
print("topic 1 :", lda_vectors[0][1])


topic 0 : 0.7292821490918172
topic 1 : 0.06685238726749793


In [ ]:
# We have the base lda - unsupervised
# Kmeans to determine number of clusters in our data
# Update LDA number of clusters to the relevant one

## K means

In [ ]:
inertias = []
ks = range(1,10)
for k in ks:
    km_test = KMeans(n_clusters=k).fit(#DATA)
    inertias.append(km_test.inertia_)
plt.plot(ks, inertias)
plt.xlabel('k cluster number')

## pre trained model

In [4]:
#Load pre trained model
model = SentenceTransformer('all-mpnet-base-v2')

/Users/martafillolbruguera/.pyenv/versions/3.10.6/envs/affinity_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
#embedding categories we want to vectorize to be able to compare to our audience

sports = model.encode("sports for young men and children")
fashion = model.encode("fashion")

#content from scraping urls

embeddings_football = model.encode("I want to start playing football")
embeddings_fashion = model.encode("I need a new outfit for the party")

In [ ]:
#compare the categories (interests from audience vs topics detected from text)
fashion @ embeddings_fashion

0.33317417

In [ ]:
topic1 = model.encode('edu game team line hockey')
topic1 @ sports

0.34371024

In [6]:
#creates dataframe with text and embedding. ¿¿Why do we need this??


df = pd.DataFrame({"texts": ["New text", "We need more sport"]})
def embed(text):
    embedding = model.encode(text)
    return embedding

df["embedding"] = df.texts.apply(embed)

In [7]:
targets = [
"Fashion",
"Pets",
"Cooking",
"Fitness",
"Movies",
"Gaming",
"Travel",
"Cars and automobiles",
"Outdoor activities",
"Books",
"Finance and investments",
"Business and entrepreneurship",
"Photography",
"Art",
"Social causes and activism",
"Health and wellness ",
"Gardening",
"Technology",
"Education and learning",
"Sports",
"Nature",
"History",
"Parenting and family",
"Music",
"Food and dining ",
"DIY and crafts ",
"Beauty",
"Science",
"Politics"
]
import pandas as pd
interests = pd.DataFrame({"interest":targets})


In [8]:
interests["embedding"] = interests.interest.apply(embed)

In [27]:
topic11 = "information medical professional"
topic11_embedding = embed(topic11)

In [28]:
topic11_embedding

array([ 2.40322389e-02, -7.10182171e-03,  1.16103375e-03, -4.24108729e-02,
       -1.16454307e-02,  1.97597761e-02,  3.98604348e-02,  7.57682770e-02,
       -2.96205096e-02,  1.71369016e-02,  2.92218439e-02, -1.93391796e-02,
        1.47796441e-02, -1.36811985e-02,  2.73945592e-02, -7.34878927e-02,
       -1.86326578e-02,  3.20309550e-02,  4.82782349e-02, -1.29107190e-02,
       -5.21783382e-02,  2.81099286e-02, -4.41021957e-02,  3.67141105e-02,
        2.91602761e-02, -3.53069678e-02,  2.80554648e-02,  3.43978847e-03,
        3.10408510e-03, -2.83938106e-02,  7.76202045e-03, -2.52980012e-02,
       -2.60417461e-02, -7.07783923e-02,  1.38531516e-06,  3.25573236e-02,
        3.13118566e-03,  3.22465338e-02, -6.64143860e-02, -7.55163208e-02,
        2.25171912e-03, -5.89696355e-02,  4.71020862e-03,  2.45765038e-02,
       -1.16063995e-04, -3.62350643e-02,  4.42391895e-02,  3.31064686e-02,
       -4.92657423e-02,  2.55532358e-02,  1.20062586e-02,  4.92227487e-02,
       -1.55193238e-02, -

In [29]:
#input_ = url # google.com
# lda topic - embed the topic
# find closest in interests embeddding
#output = interest #'Education and learning	'
results = []
for each in interests.embedding.values:
    results.append(topic11_embedding @ each)

In [31]:
import numpy as np
np.array(results).argmax()

15

In [32]:
interests

,interest,embedding
0,Fashion,"[0.01728209, 0.07585772, -0.02631025, -0.00446..."
1,Pets,"[0.045367725, 0.07620043, -0.04535109, -0.0391..."
2,Cooking,"[-0.008611952, -0.024475407, -0.0055521163, -0..."
3,Fitness,"[0.001419213, -0.0037784684, 0.01895474, 0.008..."
4,Movies,"[0.0025025068, 0.055331282, -0.0063840663, -0...."
5,Gaming,"[-0.02534862, -0.02541417, -0.046696495, 0.042..."
6,Travel,"[-0.0031953785, 0.055348586, -0.040888492, 0.0..."
7,Cars and automobiles,"[0.010719352, 0.07491442, -0.0073154974, 0.042..."
8,Outdoor activities,"[-0.013782986, -0.04851159, -0.020418737, 0.04..."
9,Books,"[0.021911902, 0.041893844, -0.0049016504, -0.0..."


In [ ]:
input_url = "webpage/results"
output_category = "Animamls"

# 100k with their interest. 100k into 20 groups. Each have their own interests. We need to find which group in interested in Animals
# We find out that this grouip is number 2
# What is the age the age of this group, gender, what are their other interests - Main other intersts Sports

# List of ads - select the best one
# Slamg in the ad
# Classic language and has sport image in
